In [1]:
import numpy as np
import torch
from ase.io import read, write
from ase import Atoms
from dpyscfl.utils import ase_atoms_to_mol, gen_mf_mol

# Case study: O3+O

In [2]:
# training dataset, O3 is trj[24], O is 20
trj = read('/home/awills/Documents/Research/sebshare/data/09/subat_ref.traj', ':')
g2o3 = trj[24]
print(g2o3.positions)
print(g2o3.info)
g2o = trj[20]
print(g2o.positions)
print(g2o.info)

[[ 0.        1.075722 -0.21567 ]
 [ 0.        0.        0.43134 ]
 [ 0.       -1.075722 -0.21567 ]]
{'O3': True, 'name': 'Ozone', 'openshell': False, 'sc': False, 'sym': False, 'n_rad': 1, 'grid_level': 1, 'pol': False, 'target_energy': -0.2345382154735096, 'energy': -225.20206708569773, 'atomization': -0.2345382154735096, 'atomization_ev': -6.382109918715892, 'atomization_H': -0.2345382154735096, 'calc_energy': -225.20206708569773, 'e_calc': -225.20206708569773}
[[0. 0. 0.]]
{'spin': 2, 'sym': False, 'grid_level': 4, 'basis': '6-311++G(3df,2pd)', 'pol': False, 'target_energy': -75.0673, 'energy': -74.99416120454062, 'atomization': -75.0673, 'atomization_ev': -2042.6852781069833, 'atomization_H': -75.0673, 'calc_energy': -75.0673, 'e_calc': -75.0673}


In [4]:
#positions from training dataset, but recreation to show how one makes the Atoms object
o3 = Atoms(['O','O','O'], positions = [[ 0.,        1.075722, -0.21567 ], 
                                       [ 0.,        0.,        0.43134 ],
                                       [ 0.,       -1.075722, -0.21567 ]])
o3.info = {'O3': True, 
           'name': 'Ozone', 
           'openshell': False, 
           'sc': False, 
           'sym': False, 
           'n_rad': 1, 
           'grid_level': 1, 
           'pol': False, 
           'target_energy': -0.2345382154735096, 
           'energy': -225.20206708569773, 
           'atomization': -0.2345382154735096, 
           'atomization_ev': -6.382109918715892, 
           'atomization_H': -0.2345382154735096, 
           'calc_energy': -225.20206708569773, 
           'e_calc': -225.20206708569773}
o = Atoms(['O'], positions=[[0,0,0]])
o.info = {'spin': 2, 
          'sym': False, 
          'grid_level': 4, 
          'basis': '6-311++G(3df,2pd)', 
          'pol': False, 
          'target_energy': -75.0673, 
          'energy': -74.99416120454062, 
          'atomization': -75.0673, 
          'atomization_ev': -2042.6852781069833, 
          'atomization_H': -75.0673, 
          'calc_energy': -75.0673, 
          'e_calc': -75.0673}

#openshell is false, 0 spin ozone -- generate the pyscf Mol objects to be used below
name_o3, mol_o3 = ase_atoms_to_mol(o3, basis='def2-tzvp', charge=0, spin=0)
name_o, mol_o = ase_atoms_to_mol(o, basis='def2-tzvp', charge=0, spin=2)

# PySCF

In [5]:
import pyscf
from pyscf import dft
print(pyscf.__file__)

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pyscf/__init__.py


In [6]:
#generate the mf and method from the mol and options
mf1_o3, method1_o3 = gen_mf_mol(mol_o3, xc='scan', pol=True, grid_level=3, nxc = False)
#run, save energy
mf1_o3.kernel()
e1_o3 = mf1_o3.e_tot

mf1_o, method1_o = gen_mf_mol(mol_o, xc='scan', pol=True, grid_level=3, nxc = False)
mf1_o.kernel()
e1_o = mf1_o.e_tot

print('PySCF (SCAN) Evaluated on:\nO3 = {}\nO = {}'.format(e1_o3, e1_o))

Building grids...
METHOD GENERATED: <function UKS at 0x7f9b3460f290>
converged SCF energy = -225.440612971224  <S^2> = 1.2447998e-10  2S+1 = 1
Single Atom -- UKS
Building grids...
METHOD GENERATED: <function UKS at 0x7f9b3460f290>
converged SCF energy = -75.0701605416889  <S^2> = 2.0048637  2S+1 = 3.0032407
PySCF (SCAN) Evaluated on:
O3 = -225.44061297122443
O = -75.0701605416889


# Pylibnxc

In [7]:
#for pylibnxc version
import pylibnxc
from pylibnxc.pyscf import RKS, UKS
print(pylibnxc.__file__)

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pylibnxc/__init__.py


In [8]:
#location of XCDiff, the network is stored within in file 'xc'
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA/xc'
#libnxc looks in cwd/MGGA_XC_CUSTOM for the xc network
try:
    os.mkdir('MGGA_XC_CUSTOM')
except:
    pass

try:
    os.symlink(modelpath, 'MGGA_XC_CUSTOM/xc')
except:
    pass

#this really just generates the method to pass into KS, since the mf is created in KS
_, method2_o3 = gen_mf_mol(mol_o3, xc='notnull', pol=True, grid_level=3, nxc=True)
#use wrapper to sent model to pylibnxc, generate and save energy
mf2_o3 = UKS(mol_o3, nxc='MGGA_XC_CUSTOM', nxc_kind='grid')
mf2_o3.kernel()
e2_o3 = mf2_o3.e_tot

_, method2_o = gen_mf_mol(mol_o, xc='notnull', pol=True, grid_level=3, nxc=True)
mf2_o = UKS(mol_o, nxc='MGGA_XC_CUSTOM', nxc_kind='grid')
mf2_o.kernel()
e2_o = mf2_o.e_tot

print('Pylibnxc (XCDiff) Evaluated on:\nO3 = {}\nO = {}'.format(e2_o3, e2_o))

Building grids...
METHOD GENERATED: functools.partial(<function KS at 0x7f9b33c9a7a0>, method=<function UKS at 0x7f9b3460f290>)
converged SCF energy = -225.170147070068  <S^2> = 9.7895025e-12  2S+1 = 1
Single Atom -- UKS
Building grids...
METHOD GENERATED: functools.partial(<function KS at 0x7f9b33c9a7a0>, method=<function UKS at 0x7f9b3460f290>)
converged SCF energy = -74.9819588245815  <S^2> = 2.004301  2S+1 = 3.0028659
Pylibnxc (XCDiff) Evaluated on:
O3 = -225.17014707006837
O = -74.9819588245815


# Dpyscfl

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [9]:
#for dpyscfl
import dpyscfl
from dpyscfl.net import get_scf
from dpyscfl.utils import old_get_datapoint
print(dpyscfl.__file__)

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/dpyscfl/__init__.py


In [10]:
#generate pytorch matrices based on mol and options
# mats_o3 = old_get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
#                                 xc='scan', zsym=o3.info.get('sym', False),
#                                 n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
#                                 init_guess=False, spin = 0,
#                                 pol=True, do_fcenter=True,
#                                 ref_path='/home/awills/Documents/Research/sebshare/data/09', ref_index= 24, ref_traj='subat_ref.traj',
#                                 ref_basis='6-311++G(3df,2pd)', dfit=False)
mats_o = old_get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 3,
                                xc='scan', zsym=o.info.get('sym', False),
                                n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
                                init_guess=False, spin = 2,
                                pol=True, do_fcenter=True,
                                ref_path='/home/awills/Documents/Research/sebshare/data/09', ref_index= 20, ref_traj='subat_ref.traj',
                                ref_basis='6-311++G(3df,2pd)', dfit=False)
#unpack returned tuples
# e_base_o3, eye_o3, mats_o3 = mats_o3
e_base_o, eye_o, mats_o = mats_o

Atoms(symbols='O', pbc=False)
6-311++G(3df,2pd)
Spin specified in atom info.
Single Atom -- UKS
Building grids...
METHOD GENERATED: <function UKS at 0x7f9b3460f290>
converged SCF energy = -75.0646782062021  <S^2> = 2.0054984  2S+1 = 3.0036634
DATAPOINT E_BASE:  -75.0646782062021
DATAPOINT MO_ENERGY:  [[-19.08788232  -0.99094106  -0.42761339  -0.42761181  -0.34420526
    0.10411979   0.16775908   0.16776322   0.17615063   0.7340606
    0.7340618    0.75850597   0.75850866   0.76865918   0.94725467
    0.94726053   0.99657217   1.15965707   3.21900344   3.21901231
    3.28032697   3.28033901   3.30517676   4.81795597   4.81797227
    4.8899124    4.95884283   4.95884312   5.01240155   5.01240306
    5.04798834   5.04799908   5.06429772  14.21084366  14.21085035
   14.27510485  14.27513692  14.30303662  49.65915519]
 [-19.02760473  -0.80391146  -0.29211552  -0.20671957  -0.20665001
    0.14272018   0.22231137   0.2224091    0.23148416   0.77396273
    0.77416728   0.79521686   0.81998582 

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pyscf/gto/moleintor.py:259: UserWarning: Function int4c1e_sph not found.  Set its comp to 1
  warnings.warn('Function %s not found.  Set its comp to 1' % intor_name)


Reference DM loaded. Shape: (2, 39, 39)
REF, NONFRAC SHAPES: DM_GUESS = (2, 39, 39), DM_BASE = (2, 39, 39)
Reading in reference target energy. Current: -75.0646782062021
Reading in reference target energy. Post-read: -75.0673
GRID GENERATION.
STATS: GRID_LEVEL=3. ZYM=False. NL_CUTOFF=0. SPIN(INP/MOL)=2/2. POL=True.
Generating spin-channel densities.
Evaluating atomic orbitals on grid.
AO Shape: (14262, 39). DM Shape: (39, 39)
Evaluating density on grid.
Evaluating atomic orbitals on grid.
AO Shape: (14262, 39). DM Shape: (39, 39)
Evaluating density on grid.
PRE-UPDATED MATRICES E_BASE, FEATURES
-75.0646782062021
-75.0673
POST-UPDATE matrices -> features
-75.0673
GET DATAPOINT MATRICES SHAPES
dm_init   ---   (2, 39, 39)
dm   ---   (2, 39, 39)
v   ---   (39, 39)
t   ---   (39, 39)
s   ---   (39, 39)
eri   ---   (39, 39, 39, 39)
n_elec   ---   ()
n_atoms   ---   1, no shape
e_nuc   ---   ()
mo_energy   ---   (2, 39)
mo_occ   ---   (2, 39)
e_ip   ---   ()
ip_idx   ---   ()
e_pretrained   -

In [11]:
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
# s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats_o['s']))

# mats_o3['s_chol'] = s_chol_o3
mats_o['s_chol'] = s_chol_o

#go through and make everything a list, since everything in the scripts assumes matrices stored as 
# key: [mat_idx0, mat_idx1, ...]
# mats_o3_l = {k: [mats_o3[k]] for k in mats_o3.keys()}
mats_o_l = {k: [mats_o[k]] for k in mats_o.keys()}

#go through and additionally make everything a tensor, since forward pass expects things to be tensors
for k in mats_o_l.keys():
    print(k)
#     o3k = mats_o3_l[k]
    ok = mats_o_l[k]
    if type(ok[0]) in [int, float]:
        print('integer/float from o3 dict')
#         mats_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not ok[0].shape:
#         mats_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
#         mats_o3_l[k] = [torch.Tensor(o3k[0])]
        mats_o_l[k] = [torch.Tensor(ok[0])]

dm_init
dm
v
t
s
eri
n_elec
n_atoms
integer/float from o3 dict
e_nuc
mo_energy
mo_occ
e_ip
ip_idx
e_pretrained
e_base
integer/float from o3 dict
dm_realinit
f_center
L
scaling
ao_eval
grid_weights
e_calc
integer/float from o3 dict
rho
s_chol


In [12]:
#location of model
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
#type MGGA specified, but additionally get_scf expects the full file, so add /xc to path
scf = get_scf('MGGA', path=modelpath+'/xc')
#do self-consistent
sc=True
#added recently -- prints brief info each pass
vvv = True
#generate results dictionary and then select energy only
# res_o3 = scf([torch.Tensor(mats_o3_l['dm_init'][0])], mats_o3_l, sc=sc, verbose=vvv)
# res_o3 = res_o3['E']

res_o = scf([torch.Tensor(mats_o_l['dm_init'][0])], mats_o_l, sc=sc, verbose=vvv)
res_o = res_o['E']

# print('Dpyscf-lite (XCDiff) Evaluated on:\nO3 = {}'.format(res_o3))
print('Dpyscf-lite (XCDiff) Evaluated on:\nO = {}'.format(res_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


Noise generation to avoid potential degeneracies
0 Energy: tensor([-74.9761], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackw

18 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<

The output of the above cell is pasted at the end of the cell in the event the generated output above gets erased.

# Dpyscf (original, contained in xcdiff folder)

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [13]:
import dpyscf
from dpyscf.utils import get_datapoint
from dpyscf.net import lite_get_scf
print(dpyscf.__file__)

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/dpyscf/__init__.py


In [14]:
ref_path = '/home/awills/Documents/Research/swxcd/aegis/test_subset_ps2/09'
#data preparation is different
#if using old scripts from xcdiff/original dpyscf,
#reference density matrices must be in ref_path/{INDEX}.dm.npy
#can just symlink the DMs produced by run_pyscf.py
#ADDITIONALLY, reference energies must be stored in ref_path/energies
#formatted as:
#IDX ENERGY
# mats2_o3 = get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
#                                 xc='scan', zsym=o3.info.get('sym', False),
#                                 n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
#                                 init_guess=False, spin = 0,
#                                 pol=True, do_fcenter=True,
#                                 ref_path=ref_path, ref_index= 24,
#                                 ref_basis='6-311++G(3df,2pd)')
mats2_o = get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 5,
                                xc='scan', zsym=o.info.get('sym', True),
                                n_rad=o.info.get('n_rad',30), n_ang=o.info.get('n_ang',15),
                                init_guess=True, spin = o.info.get('spin', 0),
                                pol=o.info.get('pol', True), do_fcenter=True,
                                ref_path=ref_path, ref_index= 20,
                                ref_basis='6-311++G(3df,2pd)')
#unpack returned tuples
# e_base2_o3, eye2_o3, mats2_o3 = mats2_o3
e_base2_o, eye2_o, mats2_o = mats2_o

Atoms(symbols='O', pbc=False)
6-311++G(3df,2pd)
Spin specified in atom info.
Single Atom -- UKS
Building grids...
METHOD GENERATED: <function UKS at 0x7f9b3460f290>


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pyscf/gto/moleintor.py:259: UserWarning: Function int4c1e_sph not found.  Set its comp to 1
  warnings.warn('Function %s not found.  Set its comp to 1' % intor_name)


SCF not converged.
SCF energy = -75.0641476440329 after 50 cycles  <S^2> = 2.0054474  2S+1 = 3.0036294
Calculation did not converge. Trying second order convergence with PBE to feed into calculation.
converged SCF energy = -75.0045322567339  <S^2> = 2.0031221  2S+1 = 3.0020807
PBE Calculation complete -- feeding into original kernel.
converged SCF energy = -75.0641705575444  <S^2> = 2.0054759  2S+1 = 3.0036484
Loading reference density


In [15]:
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
# s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats2_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats2_o['s']))

#old version expects this under different key names
# mats2_o3['s_chol'] = s_chol_o3
# mats2_o3['s_oh'] = s_chol_o3
# mats2_o3['s_inv_oh'] = s_chol_o3.T

mats2_o['s_chol'] = s_chol_o
mats2_o['s_oh'] = s_chol_o
mats2_o['s_inv_oh'] = s_chol_o.T

#go through and make everything a list
# mats2_o3_l = {k: [mats2_o3[k]] for k in mats2_o3.keys()}
mats2_o_l = {k: [mats2_o[k]] for k in mats2_o.keys()}

#go through and additionally make everything a tensor
for k in mats2_o_l.keys():
    print(k)
#     o3k = mats2_o3_l[k]
    ok = mats2_o_l[k]
    if type(ok[0]) in [int, float]:
        print('integer/float from o3 dict')
#         mats2_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats2_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not ok[0].shape:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats2_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0])]
        mats2_o_l[k] = [torch.Tensor(ok[0])]

dm_init
v
t
s
eri
n_elec
n_atoms
e_nuc
mo_energy
mo_occ
e_ip
ip_idx
e_pretrained
L
scaling
rho
ao_eval
grid_weights
fcenter
s_chol
s_oh
s_inv_oh


In [16]:
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
scf2 = lite_get_scf('MGGA', path=modelpath+'/xc')

sc=True
vvv=True

# res2_o3 = scf2([torch.Tensor(mats2_o3_l['dm_init'][0])], mats2_o3_l, sc=sc, verbose=vvv)
# res2_o3 = res2_o3['E']

res2_o = scf2([torch.Tensor(mats2_o_l['dm_init'][0])], mats2_o_l, sc=sc, verbose=vvv)
res2_o = res2_o['E']

# print('Dpyscf (XCDiff) Evaluated on:\nO3 = {}'.format(res2_o3))
print('Dpyscf (XCDiff) Evaluated on:\nO = {}'.format(res2_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)


0 Energy: tensor([-74.8719], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-43.0739], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-34.1753], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>), tensor([-34.1753], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-29.3438], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>), tensor([-34.1753], grad_fn=<AddBackward0>), tensor([-29.3438], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-26.1585], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>), tensor([-34.1753], grad_fn=<AddBackward0>), tensor([-29.3438], grad_fn=<AddBackward0>), tensor([-26.1585], grad_fn=<AddBackward0>

18 Energy: tensor([-18.1652], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<AddBackward0>), tensor([-43.0739], grad_fn=<AddBackward0>), tensor([-34.1753], grad_fn=<AddBackward0>), tensor([-29.3438], grad_fn=<AddBackward0>), tensor([-26.1585], grad_fn=<AddBackward0>), tensor([-23.9388], grad_fn=<AddBackward0>), tensor([-22.3630], grad_fn=<AddBackward0>), tensor([-21.2419], grad_fn=<AddBackward0>), tensor([-20.4224], grad_fn=<AddBackward0>), tensor([-19.8202], grad_fn=<AddBackward0>), tensor([-19.3662], grad_fn=<AddBackward0>), tensor([-19.0277], grad_fn=<AddBackward0>), tensor([-18.7535], grad_fn=<AddBackward0>), tensor([-18.5708], grad_fn=<AddBackward0>), tensor([-18.4520], grad_fn=<AddBackward0>), tensor([-18.3736], grad_fn=<AddBackward0>), tensor([-18.3333], grad_fn=<AddBackward0>), tensor([-18.2349], grad_fn=<AddBackward0>), tensor([-18.1652], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-18.1285], grad_fn=<AddBackward0>)
History: [tensor([-74.8719], grad_fn=<

# Dpyscf (original, contained in standalone repo folder, hosted [here](https://github.com/semodi/dpyscf))

**WARNING: On my machine at least, PyTorch *does not release the memory used during an SCF cycle.* Doing the above calculations with 25 SCF steps uses ~50 GB of RAM, which might not get released until restarting this notebook's kernel. User beware.**

In [2]:
#for original+xcdiff dpyscf
import ogdpyscf
from ogdpyscf.utils import get_datapoint as og_get_datapoint
from ogdpyscf.net import lite_get_scf
print(ogdpyscf.__file__)

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/ogdpyscf/__init__.py


In [5]:
ref_path = '/home/awills/Documents/Research/swxcd/aegis/test_subset_ps2/09'
#data preparation is different
#if using old scripts from xcdiff/original dpyscf,
#reference density matrices must be in ref_path/{INDEX}.dm.npy
#can just symlink the DMs produced by run_pyscf.py
#ADDITIONALLY, reference energies must be stored in ref_path/energies
#formatted as:
#IDX ENERGY
# mats2_o3 = get_datapoint(o3, basis='6-311++G(3df,2pd)', grid_level= 3,
#                                 xc='scan', zsym=o3.info.get('sym', False),
#                                 n_rad=o3.info.get('n_rad',30), n_ang=o3.info.get('n_ang',15),
#                                 init_guess=False, spin = 0,
#                                 pol=True, do_fcenter=True,
#                                 ref_path=ref_path, ref_index= 24,
#                                 ref_basis='6-311++G(3df,2pd)')
mats2_o = og_get_datapoint(o, basis='6-311++G(3df,2pd)', grid_level= 5,
                                xc='scan', zsym=o.info.get('sym', True),
                                n_rad=o.info.get('n_rad',30), n_ang=o.info.get('n_ang',15),
                                init_guess=True, spin = o.info.get('spin', 0),
                                pol=o.info.get('pol', True), do_fcenter=True,
                                ref_path=ref_path, ref_index= 20,
                                ref_basis='6-311++G(3df,2pd)')
#unpack returned tuples
# e_base2_o3, eye2_o3, mats2_o3 = mats2_o3
e_base2_o, eye2_o, mats2_o = mats2_o

Atoms(symbols='O', pbc=False)
6-311++G(3df,2pd)


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/pyscf/gto/moleintor.py:259: UserWarning: Function int4c1e_sph not found.  Set its comp to 1
  warnings.warn('Function %s not found.  Set its comp to 1' % intor_name)


converged SCF energy = -75.0641994644861  <S^2> = 2.0054523  2S+1 = 3.0036327
Loading reference density
converged SCF energy = -75.0641194822574  <S^2> = 2.0054504  2S+1 = 3.0036314


In [6]:
#need to generate more matrices for network -- this is usually handled by the data reader class used in training
# s_chol_o3 = np.linalg.inv(np.linalg.cholesky(mats2_o3['s']))
s_chol_o = np.linalg.inv(np.linalg.cholesky(mats2_o['s']))

#old version expects this under different key names
# mats2_o3['s_chol'] = s_chol_o3
# mats2_o3['s_oh'] = s_chol_o3
# mats2_o3['s_inv_oh'] = s_chol_o3.T

mats2_o['s_chol'] = s_chol_o
mats2_o['s_oh'] = s_chol_o
mats2_o['s_inv_oh'] = s_chol_o.T

#go through and make everything a list
# mats2_o3_l = {k: [mats2_o3[k]] for k in mats2_o3.keys()}
mats2_o_l = {k: [mats2_o[k]] for k in mats2_o.keys()}

#go through and additionally make everything a tensor
for k in mats2_o_l.keys():
    print(k)
#     o3k = mats2_o3_l[k]
    ok = mats2_o_l[k]
    if type(ok[0]) in [int, float]:
        print('integer/float from o3 dict')
#         mats2_o3_l[k] = [torch.Tensor(np.array(o3k))]
        mats2_o_l[k] = [torch.Tensor(np.array(ok))]
    elif not ok[0].shape:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0].reshape(1,))]
        mats2_o_l[k] = [torch.Tensor(ok[0].reshape(1,))]
    else:
#         mats2_o3_l[k] = [torch.Tensor(o3k[0])]
        mats2_o_l[k] = [torch.Tensor(ok[0])]

dm_init
v
t
s
n_elec
n_atoms
e_nuc
mo_energy
mo_occ
e_ip
ip_idx
e_pretrained
eri
L
scaling
rho
ao_eval
grid_weights
fcenter
s_chol
s_oh
s_inv_oh


In [7]:
modelpath = '/home/awills/Documents/Research/dpyscfl/models/xcdiff/MODEL_MGGA'
scf3 = lite_get_scf('MGGA', path=modelpath+'/xc')

sc=True
vvv=True

# res2_o3 = scf3([torch.Tensor(mats2_o3_l['dm_init'][0])], mats2_o3_l, sc=sc, verbose=vvv)
# res2_o3 = res2_o3['E']

res2_o = scf3([torch.Tensor(mats2_o_l['dm_init'][0])], mats2_o_l, sc=sc, verbose=vvv)
res2_o = res2_o['E']

# print('Dpyscf (Standalone Repo) Evaluated on:\nO3 = {}'.format(res2_o3))
print('Dpyscf (Standalone Repo) Evaluated on:\nO = {}'.format(res2_o))

FREEC False
SCF Loop Beginning: 25 Steps
Step 0


/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)
/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/ogdpyscf/torch_routines.py:77: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2524.)
  e, c = torch.symeig(co

0 Energy: tensor([-74.8328], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-74.8646], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-74.8862], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-74.9006], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>), tensor([-74.9006], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-74.9111], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>), tensor([-74.9006], grad_fn=<AddBackward0>), tensor([-74.9111], grad_fn=<AddBackward0>

18 Energy: tensor([-74.9383], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<AddBackward0>), tensor([-74.8646], grad_fn=<AddBackward0>), tensor([-74.8862], grad_fn=<AddBackward0>), tensor([-74.9006], grad_fn=<AddBackward0>), tensor([-74.9111], grad_fn=<AddBackward0>), tensor([-74.9187], grad_fn=<AddBackward0>), tensor([-74.9244], grad_fn=<AddBackward0>), tensor([-74.9284], grad_fn=<AddBackward0>), tensor([-74.9314], grad_fn=<AddBackward0>), tensor([-74.9334], grad_fn=<AddBackward0>), tensor([-74.9349], grad_fn=<AddBackward0>), tensor([-74.9360], grad_fn=<AddBackward0>), tensor([-74.9367], grad_fn=<AddBackward0>), tensor([-74.9373], grad_fn=<AddBackward0>), tensor([-74.9376], grad_fn=<AddBackward0>), tensor([-74.9379], grad_fn=<AddBackward0>), tensor([-74.9381], grad_fn=<AddBackward0>), tensor([-74.9382], grad_fn=<AddBackward0>), tensor([-74.9383], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-74.9383], grad_fn=<AddBackward0>)
History: [tensor([-74.8328], grad_fn=<

# Pasted Outputs

## Dpyscfl Output

In case the output gets erased for some reason and the above cell can't be run, the output is:
```
FREEC False
SCF Loop Beginning: 25 Steps
Step 0
Noise generation to avoid potential degeneracies
0 Energy: tensor([-225.1551], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-225.1555], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-225.1556], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
Step 5
5 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 17

17 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 18
18 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-225.1558], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-225.1557], grad_fn=<AddBackward0>)
History: [tensor([-225.1551], grad_fn=<AddBackward0>), tensor([-225.1555], grad_fn=<AddBackward0>), tensor([-225.1556], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1558], grad_fn=<AddBackward0>), tensor([-225.1557], grad_fn=<AddBackward0>)]
SCF Loop Beginning: 25 Steps
Step 0

Noise generation to avoid potential degeneracies
0 Energy: tensor([-74.9761], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-74.9764], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 5
5 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 17
17 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 18

18 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-74.9765], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9764], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>), tensor([-74.9765], grad_fn=<AddBackward0>)]
Dpyscf-lite (XCDiff) Evaluated on:
O3 = tensor([-225.1551, -225.1555, -225.1556, -225.1557, -225.1557, -225.1558,
        -225.1558, -225.1558, -225.1558, -225.1558, -225.1558, -225.1558,
        -225.1558, -225.1558, -225.1558, -225.1558, -225.1558, -225.1558,
        -225.1558, -225.1558, -225.1558, -225.1558, -225.1558, -225.1558,
        -225.1557], grad_fn=<CatBackward0>)
O = tensor([-74.9761, -74.9764, -74.9764, -74.9765, -74.9765, -74.9765, -74.9765,
        -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765,
        -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765, -74.9765,
        -74.9765, -74.9765, -74.9765, -74.9765], grad_fn=<CatBackward0>)

```

## Original Dpyscf Output

In case the output gets erased for some reason and the above cell can't be run, the output is:
```
FREEC False

/home/awills/anaconda3/envs/rsrch/lib/python3.7/site-packages/torch/serialization.py:709: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  " silence this warning)", UserWarning)

SCF Loop Beginning: 25 Steps
Step 0
0 Energy: tensor([-225.1550], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-118.4661], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-89.9705], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-76.8356], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-68.9395], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>)]
Step 5
5 Energy: tensor([-63.6487], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-60.0146], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-57.4984], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-55.7549], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-54.5512], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-53.7476], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-53.2192], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-52.8860], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-52.6544], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-52.5047], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-52.4044], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-52.3478], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>)]
Step 17
17 Energy: tensor([-52.3053], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>)]
Step 18

18 Energy: tensor([-52.2767], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>)]
Step 19
19 Energy: tensor([-52.2593], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-52.2522], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-52.2416], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-52.2417], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>), tensor([-52.2417], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-52.2341], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>), tensor([-52.2417], grad_fn=<AddBackward0>), tensor([-52.2341], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-52.2365], grad_fn=<AddBackward0>)
History: [tensor([-225.1550], grad_fn=<AddBackward0>), tensor([-118.4661], grad_fn=<AddBackward0>), tensor([-89.9705], grad_fn=<AddBackward0>), tensor([-76.8356], grad_fn=<AddBackward0>), tensor([-68.9395], grad_fn=<AddBackward0>), tensor([-63.6487], grad_fn=<AddBackward0>), tensor([-60.0146], grad_fn=<AddBackward0>), tensor([-57.4984], grad_fn=<AddBackward0>), tensor([-55.7549], grad_fn=<AddBackward0>), tensor([-54.5512], grad_fn=<AddBackward0>), tensor([-53.7476], grad_fn=<AddBackward0>), tensor([-53.2192], grad_fn=<AddBackward0>), tensor([-52.8860], grad_fn=<AddBackward0>), tensor([-52.6544], grad_fn=<AddBackward0>), tensor([-52.5047], grad_fn=<AddBackward0>), tensor([-52.4044], grad_fn=<AddBackward0>), tensor([-52.3478], grad_fn=<AddBackward0>), tensor([-52.3053], grad_fn=<AddBackward0>), tensor([-52.2767], grad_fn=<AddBackward0>), tensor([-52.2593], grad_fn=<AddBackward0>), tensor([-52.2522], grad_fn=<AddBackward0>), tensor([-52.2416], grad_fn=<AddBackward0>), tensor([-52.2417], grad_fn=<AddBackward0>), tensor([-52.2341], grad_fn=<AddBackward0>), tensor([-52.2365], grad_fn=<AddBackward0>)]
SCF Loop Beginning: 25 Steps
Step 0
0 Energy: tensor([-74.9761], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>)]
Step 1
1 Energy: tensor([-43.2015], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>)]
Step 2
2 Energy: tensor([-34.2806], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>)]
Step 3
3 Energy: tensor([-29.4604], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>)]
Step 4
4 Energy: tensor([-26.3482], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>)]
Step 5

5 Energy: tensor([-24.1357], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>)]
Step 6
6 Energy: tensor([-22.5100], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>)]
Step 7
7 Energy: tensor([-21.3647], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>)]
Step 8
8 Energy: tensor([-20.5265], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>)]
Step 9
9 Energy: tensor([-19.8936], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>)]
Step 10
10 Energy: tensor([-19.4032], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>)]
Step 11
11 Energy: tensor([-19.0713], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>)]
Step 12
12 Energy: tensor([-18.8011], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>)]
Step 13
13 Energy: tensor([-18.6086], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>)]
Step 14
14 Energy: tensor([-18.4598], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>)]
Step 15
15 Energy: tensor([-18.3561], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>)]
Step 16
16 Energy: tensor([-18.2716], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>)]
Step 17
17 Energy: tensor([-18.1856], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>)]
Step 18
18 Energy: tensor([-18.0978], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>)]
Step 19

19 Energy: tensor([-18.0366], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>)]
Step 20
20 Energy: tensor([-18.0095], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>)]
Step 21
21 Energy: tensor([-17.9769], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>)]
Step 22
22 Energy: tensor([-17.9236], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>), tensor([-17.9236], grad_fn=<AddBackward0>)]
Step 23
23 Energy: tensor([-17.8774], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>), tensor([-17.9236], grad_fn=<AddBackward0>), tensor([-17.8774], grad_fn=<AddBackward0>)]
Step 24
24 Energy: tensor([-17.8375], grad_fn=<AddBackward0>)
History: [tensor([-74.9761], grad_fn=<AddBackward0>), tensor([-43.2015], grad_fn=<AddBackward0>), tensor([-34.2806], grad_fn=<AddBackward0>), tensor([-29.4604], grad_fn=<AddBackward0>), tensor([-26.3482], grad_fn=<AddBackward0>), tensor([-24.1357], grad_fn=<AddBackward0>), tensor([-22.5100], grad_fn=<AddBackward0>), tensor([-21.3647], grad_fn=<AddBackward0>), tensor([-20.5265], grad_fn=<AddBackward0>), tensor([-19.8936], grad_fn=<AddBackward0>), tensor([-19.4032], grad_fn=<AddBackward0>), tensor([-19.0713], grad_fn=<AddBackward0>), tensor([-18.8011], grad_fn=<AddBackward0>), tensor([-18.6086], grad_fn=<AddBackward0>), tensor([-18.4598], grad_fn=<AddBackward0>), tensor([-18.3561], grad_fn=<AddBackward0>), tensor([-18.2716], grad_fn=<AddBackward0>), tensor([-18.1856], grad_fn=<AddBackward0>), tensor([-18.0978], grad_fn=<AddBackward0>), tensor([-18.0366], grad_fn=<AddBackward0>), tensor([-18.0095], grad_fn=<AddBackward0>), tensor([-17.9769], grad_fn=<AddBackward0>), tensor([-17.9236], grad_fn=<AddBackward0>), tensor([-17.8774], grad_fn=<AddBackward0>), tensor([-17.8375], grad_fn=<AddBackward0>)]
Dpyscf (XCDiff) Evaluated on:
O3 = tensor([-225.1550, -118.4661,  -89.9705,  -76.8356,  -68.9395,  -63.6487,
         -60.0146,  -57.4984,  -55.7549,  -54.5512,  -53.7476,  -53.2192,
         -52.8860,  -52.6544,  -52.5047,  -52.4044,  -52.3478,  -52.3053,
         -52.2767,  -52.2593,  -52.2522,  -52.2416,  -52.2417,  -52.2341,
         -52.2365], grad_fn=<CatBackward0>)
O = tensor([-74.9761, -43.2015, -34.2806, -29.4604, -26.3482, -24.1357, -22.5100,
        -21.3647, -20.5265, -19.8936, -19.4032, -19.0713, -18.8011, -18.6086,
        -18.4598, -18.3561, -18.2716, -18.1856, -18.0978, -18.0366, -18.0095,
        -17.9769, -17.9236, -17.8774, -17.8375], grad_fn=<CatBackward0>)
```